##### This module runs the ECG algorithm, tailored to the TimeWak datasets


In [1]:
import numpy as np
import pandas as pd

import ECG.ECG_parameters as param
from ECG.ECG_fragile import SignalProcessing, WatermarkGenerator, FragileWatermark, SignalAnalysis2
from ECG.ECG_robust import Preprocessing, WatermarkEmbedding, SignalAnalysis
from utils import get_mae, normalize_dataset

beeeeep


In [2]:
"""From TimeWak. load the real and fake data"""
iterations = 5

original_data = pd.read_csv('datasets/ETTh.csv', 
                            parse_dates=[0], 
                            date_format= '%d-%m-%y %H:%M')


In [3]:
"""Function for Robust embedding for 1 timeseries"""

def run_robust_ECG_for_1_column(column_number):
    """Run the ECG watermark for 1 column. NOTE we normalize the signal y-vals before watermarking"""
    binary_ssn       = Preprocessing.convert_ssn_to_binary(param.user_ssn)
    binary_ssn_split = Preprocessing.split_and_pad_binary_ssn(binary_ssn, binary_ssn_chunk_len = 4)
    ssn_with_hamming = Preprocessing.apply_hamming_to_all_ssn_chunks(binary_ssn_split)

    subsequence_length = param.subsequence_len_factor * len(ssn_with_hamming) # m = 3*l (from paper)

    # n_timesteps = math.floor(subsequence_length * param.num_subsequences) # from paper
    n_timesteps = len(original_data.iloc[:, 0]) # how many x-values
    signal = normalize_dataset(original_data.iloc[:, column_number].values)

    watermark_sequence = WatermarkEmbedding._turn_watermark_to_nonbinary_sequence(ssn_with_hamming)
    ecg_subsequences   = WatermarkEmbedding._split_signal_to_subsequences(signal, subsequence_length, n_timesteps)
    watermarked_signal = WatermarkEmbedding.get_watermarked_subsequences(ecg_subsequences, watermark_sequence)

    mae = get_mae(signal, watermarked_signal)
    return signal, watermarked_signal, mae



In [4]:
"""Run the single-timeseries ROBUST embedding algo on ALL timeseries (except for x-axis)"""

num_columns = original_data.shape[1]

mae_list = []
for col_num in range(1, num_columns): # start at col 1 to ignore x-axis
    signal, watermarked_signal, mae = run_robust_ECG_for_1_column(col_num)
    mae_list.append(mae)

print(f"Robust: MAE {np.mean(mae_list)},\n std={np.std(mae_list)}")


# should_we_plot = 0
# SignalAnalysis.plot_robust_results(should_we_plot, signal, watermarked_signal)

Robust: MAE 0.0002520403628882123,
 std=6.574687213604422e-07


In [5]:
"""Fragile embedding. NOTE the input to this is the ROBUSTLY WATERMARKED signal"""

def run_fragile_ECG_for_1_column(watermarked_signal):
    """Input: the output of the robust watermark algo"""

    # shifted_ecg_signal, min_value   = SignalProcessing.shift_signal_up_to_remove_negative_values(robust.ecg_signal)
    shifted_ecg_signal, min_value   = SignalProcessing.shift_signal_up_to_remove_negative_values(watermarked_signal)
    scaled_signal                   = SignalProcessing.scale_signal_and_remove_decimals(shifted_ecg_signal, param.ECG_SCALE_FACTOR)
    scaled_signal_no_lsb            = SignalProcessing.remove_lsb_from_each_element_in_signal(scaled_signal)
    segments_list, num_segments_in_signal= WatermarkGenerator.split_signal_to_heartbeat_segments(scaled_signal_no_lsb)
    window_indices_for_all_segments = WatermarkGenerator.get_window_indices_for_all_segments(segments_list, param.SEED_K)
    segment_hashes                  = FragileWatermark.compute_segment_power_hashes(scaled_signal_no_lsb, window_indices_for_all_segments, num_segments_in_signal)
    print(f"scaled_signal_no_lsb shape: {scaled_signal_no_lsb.shape}")
    print(f"Number of segments: {num_segments_in_signal}")
    print(f"First segment indices: {window_indices_for_all_segments[0] if window_indices_for_all_segments else 'Empty'}")
    quantized_segment_hashes        = FragileWatermark.quantize_hash_values_for_all_segments(segment_hashes, param.BIT_LENGTH)
    seeded_hash_segments            = FragileWatermark.prepend_seed_to_every_hash(quantized_segment_hashes, param.SEED_K, param.BIT_LENGTH)
    print(f"First few seeded hashes: {seeded_hash_segments[:5]}")
    watermarks_for_all_segments     = FragileWatermark.convert_hash_to_int_and_generate_watermark(segments_list, seeded_hash_segments)
    # watermarked_signal              = embed_watermark_into_ecg(scaled_signal_no_lsb, segments_list, watermarks_for_all_segments)
    watermarked_ecg_segments        = FragileWatermark.apply_lsb_watermark_to_ecg_segments(scaled_signal_no_lsb, segments_list, watermarks_for_all_segments)
    watermarked_signal              = FragileWatermark.concat_watermarked_segments(watermarked_ecg_segments)
    watermarked_ecg_signal_unscaled = SignalProcessing.unscale_signal(watermarked_signal, param.ECG_SCALE_FACTOR)
    watermarked_ecg_signal_unshifted= SignalProcessing.unshift_signal_back_to_original(watermarked_ecg_signal_unscaled, min_value)

    return watermarked_ecg_signal_unshifted

watermarked_ecg_signal_unshifted = run_fragile_ECG_for_1_column(watermarked_signal)

fragile_mae = get_mae(signal, watermarked_ecg_signal_unshifted)
print(f"Fragile MAE: {fragile_mae}")

should_we_plot = 0
SignalAnalysis2.plot_fragile_results(should_we_plot, watermarked_ecg_signal_unshifted)


scaled_signal_no_lsb shape: (17420,)
Number of segments: 272
First segment indices: [array([15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
       32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]), array([ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35])]
Seeded hash example: 0000000000000000001011111111111111111111
First few seeded hashes: ['0000000000000000001011111111111111111111', '0000000000000000001011111111111111111111', '0000000000000000001011111111111111111111', '0000000000000000001011111111111111111111', '0000000000000000001011111111111111111111']
Fragile MAE: 0.0008166295453456023


c:\watermarking\ECG\ECG_fragile.py:198: RuntimeWarning: invalid value encountered in cast
  scaled_values = (segment_hashes * (2**bit_length - 1)).astype(int)  # Scale to [0, 255] for 8-bit
